# DRL usage example

In this notebook example, Stable Baselines 3 has been used to train and to load an agent. However, Sinergym is completely agnostic to any DRL algorithm (although there are custom callbacks for SB3 specifically) and can be used with any DRL library that works with gymnasium environments.

## Training a model

We are going to rely on the script available in the repository root called `DRL_battery.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning algorithms and set parameters to everything so that we can define the training options from the execution of the script easily by a JSON file.

For more information about how run `DRL_battery.py`, please, see [Train a model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#train-a-model).

In [1]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat, Logger
from stable_baselines3.common.monitor import Monitor

import sinergym
import sinergym.utils.gcloud as gcloud
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import CSVLogger, WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

First let's define some variables for the execution.

In [2]:
# Environment ID
environment = "Eplus-demo-v1"
# Training episodes
episodes = 4
#Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'SB3_DQN-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date

We can combine this experiment executions with [Weights&Biases](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#weights-and-biases-structure) in order to host all information extracted. With *wandb*, it’s possible to track and visualize all DRL training process in real time, register hyperparameters and details of each experiment, save artifacts such as models and *sinergym* output, and compare between different executions.

In [3]:
# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alex_ugr. Use `wandb login --relogin` to force relogin


Now we are ready to create the Gymnasium Environment. Here we use the environment name defined, remember that you can [change default environment configuration](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/change_environment.html#Changing-an-environment-registered-in-Sinergym). We will create a eval_env too in order to interact in the evaluation episodes. We can overwrite the env name with experiment name if we want.

In [4]:
env = gym.make(environment, env_name=experiment_name)
eval_env = gym.make(environment, env_name=experiment_name+'_EVALUATION')

[2023-05-26 08:32:44,154] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:32:44,156] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:32:44,157] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-26 08:32:44,158] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-05-26 08:32:44,158] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Setting up action definition in building model if exists...
[2023-05-26 08:32:44,212] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:3

We can also add a Wrapper to the environment, we are going to use a Logger (extension of `gym.Wrapper`) this is used to monitor and log the interactions with the environment and save the data into a CSV. Files generated will be stored as artifact in *wandb* too.

In [5]:
env = LoggerWrapper(env)

At this point, we have the environment set up and ready to be used. We are going to create our learning model (Stable Baselines 3 DQN), but we can use any other algorithm.

In [6]:
model = DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Now we need to calculate the number of timesteps of each episode for the evaluation. Evaluation will execute the current model during a number of episodes determined to decide if it is the best current version of the model at that point of the training. Output generated will be stored in *wandb* server too.

In [7]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize


We are going to use the LoggerEval callback to print and save the best model evaluated during training.

In [8]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    eval_env,
    best_model_save_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    log_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=1)
callbacks.append(eval_callback)

callback = CallbackList(callbacks)

In order to track all the training process in *wandb*, it is necessary to create a callback with a compatible wandb output format (which call *wandb* log method in the learning algorithm process).

In [9]:
# wandb logger and setting in SB3
logger = Logger(
    folder=None,
    output_formats=[
        HumanOutputFormat(
            sys.stdout,
            max_length=120),
        WandBOutputFormat()])
model.set_logger(logger)
# Append callback
log_callback = LoggerCallback()
callbacks.append(log_callback)


callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [10]:
timesteps = episodes * n_timesteps_episode

Now is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [11]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

[2023-05-26 08:32:44,713] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:32:44,906] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:33:02,051] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:33:02,052] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:33:02,143] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32-res1/Eplus-env-sub_run2
------------------------------------------------------------------------------------
| action/                                                             |            |
|    Cooling_Setpoint_RL                                              | 25.5       |
|    Heating_Setpoint_RL                                              | 19.1       |
| action_simulation/                                                  |            |
|    Cooling_Se

Now, we save the current model (model version when training has finished).

In [12]:
model.save(env.simulator._env_working_dir_parent + '/' + experiment_name)

And as always, remember to close the environment.

In [13]:
env.close()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2023-05-26 08:35:26,449] EPLUS_ENV_SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


We have to upload all Sinergym output as *wandb* artifact. This output include all sinergym_output (and LoggerWrapper CSV files) and models generated in training and evaluation episodes.

In [14]:
artifact = wandb.Artifact(
        name="training",
        type="experiment1")
artifact.add_dir(
        env.simulator._env_working_dir_parent,
        name='training_output/')
artifact.add_dir(
    eval_env.simulator._env_working_dir_parent,
    name='evaluation_output/')
run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32-res1)... Done. 0.1s
wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-Eplus-demo-v1-episodes-4_2023-05-26_08:32_EVALUATION-res1)... Done. 0.1s


action/Cooling_Setpoint_RL,█▂▁▂
action/Heating_Setpoint_RL,▁▇█▇
action_simulation/Cooling_Setpoint_RL,█▂▁▂
action_simulation/Heating_Setpoint_RL,▁▇█▇
episode/comfort_violation_time(%),█▇▁▁
episode/cumulative_comfort_penalty,▁▆█▇
episode/cumulative_power,▁▅██
episode/cumulative_power_penalty,█▄▁▁
episode/cumulative_reward,▁▆█▇
episode/ep_length,▁▁▁▁
episode/mean_comfort_penalty,▁▆█▇


We have all the experiments results in our local computer, but we can see the execution in *wandb* too:


- If we check our projects, we can see the execution allocated:

![wandb_projects1](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_projects1.png?raw=true)


- Hyperparameters tracked in the training experiment:

![wandb_training_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_hyperparameters.png?raw=true)



- Artifacts registered (if evaluation is enabled, best model is registered too):
  
![wandb_training_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_artifact.png?raw=true)



- Visualization of metrics in real time:
  
![wandb_training_charts](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_charts.png?raw=true)

## Loading a model

We are going to rely on the script available in the repository root called `load_agent.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning models loaded and set parameters to everything so that we can define the load options from the execution of the script easily by a JSON file.

For more information about how run `load_agent.py`, please, see [Load a trained model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#load-a-trained-model).

First we define the Sinergym environment ID where we want to check the loaded agent and the name of the evaluation experiment.

In [15]:
# Environment ID
environment = "Eplus-demo-v1"
# Episodes
episodes=5
# Evaluation name
evaluation_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
evaluation_name = 'SB3_DQN-EVAL-' + environment + \
    '-episodes-' + str(episodes)
evaluation_name += '_' + evaluation_date

We can also use *wandb* here. We can allocate this evaluation of a loaded model in other project in order to not merge experiments. 

In [16]:

# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym_evaluations',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

We make the gymnasium environment and wrap with LoggerWrapper. We can use the evaluation experiment name to rename the environment.

In [17]:
env=gym.make(environment, env_name=evaluation_name)
env=LoggerWrapper(env)

[2023-05-26 08:35:44,274] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:35:44,275] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:35:44,277] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-26 08:35:44,277] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-05-26 08:35:44,278] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Setting up action definition in building model if exists...


We load the Stable Baselines 3 DQN model using the model allocated in our local computer, although we can use a remote model allocated in *wandb* from other training experiment.

In [18]:
# get wandb artifact path (to load model)
load_artifact_entity = 'alex_ugr'
load_artifact_project = 'sinergym'
load_artifact_name = 'training'
load_artifact_tag = 'latest'
load_artifact_model_path = 'evaluation_output/best_model/model.zip'
wandb_path = load_artifact_entity + '/' + load_artifact_project + \
    '/' + load_artifact_name + ':' + load_artifact_tag
# Download artifact
artifact = run.use_artifact(wandb_path)
artifact.get_path(load_artifact_model_path).download('.')
# Set model path to local wandb file downloaded
model_path = './' + load_artifact_model_path
model = DQN.load(model_path)

As we can see, The *wandb* model we want to load can come from an artifact of an different entity or project from the one we are using to register the evaluation of the loaded model, as long as it is accessible.
The next step is use the model to predict actions and interact with the environment in order to collect data to evaluate the model.

In [19]:
for i in range(episodes):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:
            current_month = info['month']
            print(info['month'], sum(rewards))
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

[2023-05-26 08:35:45,677] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:35:45,812] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


1 -0.830348444547302
2 -1730.6829453012092
3 -3545.7671945926318
4 -4692.961008551935
5 -5469.236536607371
6 -6235.107970886588
7 -8528.284908692733
8 -10984.995142143025
9 -13428.543608769201
10 -15618.815441406427
11 -16695.779938547083
12 -17668.211895692
1 -19226.671112311986
Episode  0 Mean reward:  -0.5487063673605038 Cumulative reward:  -19226.671112311986
[2023-05-26 08:36:03,143] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:36:03,144] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:36:03,257] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35-res1/Eplus-env-sub_run2
1 -0.830348444547302
2 -1747.1947558564166
3 -3579.724

Finally, we register the evaluation data in wandb as an artifact to save it.

In [20]:
artifact = wandb.Artifact(
    name="evaluating",
    type="evaluation1")
artifact.add_dir(
    env.simulator._env_working_dir_parent,
    name='evaluation_output/')

run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-05-26_08:35-res1)... Done. 0.2s


We have the loaded model results in our local computer, but we can see the execution in *wandb* too:

- If we check the wandb project list, we can see that sinergym_evaluations project has a new run:

![wandb_project2](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_project2.png?raw=true)


- Hyperparameters tracked in the evaluation experiment and we can see the previous training artifact used to load the model:

![wandb_evaluating_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_hyperparameters.png?raw=true)



- Artifact registered with Sinergym Output (and CSV files generated with the Logger Wrapper):
  
![wandb_evaluating_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_artifact.png?raw=true)